## Introduction

In this tutorial we will demonstrate that connecting to remote devices is as easy as connecting to local devices. This notebook is part of a docker compose environment that spins a remote 
microscope camera and a remote light source. These remote devices are trigered by running, on a remote machine, a python module that takes a python script defining a ```DEVICES``` list of all the devices that you want to serve, the class of the device and the host and port where the device will be served.

```python
# server_conf_camera_0.py
import Pyro4

from microscope.device_server import device
from microscope.simulators import SimulatedCamera

Pyro4.config.COMPRESSION = True
Pyro4.config.PICKLE_PROTOCOL_VERSION = 2

DEVICES = [
    device(SimulatedCamera, host="camera_0", port=8000),
]
```

Then you can run the module with the following command

```bash
python -m microscope.device_server server_conf_camera_0.py
```

In the log of the terminal you should see something like this

```bash
2023-11-12 19:29:47 2023-11-12 18:29:47,937:device-server (__main__):INFO:PID 1: Device Server started. Press Ctrl+C to exit.
2023-11-12 19:29:47 2023-11-12 18:29:47,958:SimulatedCamera (__main__):INFO:PID 14: Device initialized; starting daemon.
2023-11-12 19:29:47 2023-11-12 18:29:47,958:SimulatedCamera (__main__):INFO:PID 14: Serving PYRO:SimulatedCamera@camera_0:8000
```

Similarly, you can run a light source server with the following command

```bash
python -m microscope.device_server server_conf_light_0.py
```

where the ```server_conf_light_0.py``` file is

```python
import Pyro4

from microscope.device_server import device
from microscope.simulators import SimulatedLightSource

Pyro4.config.COMPRESSION = True
Pyro4.config.PICKLE_PROTOCOL_VERSION = 2


DEVICES = [
    device(SimulatedLightSource, host="light_0", port=9000),
]
```


## Connecting to devices

To connect any device, first you have to import Pyro4. Pyro4 is the library we use to connect to remote devices. Then we need to settings for Pyro4 to be able to connect. These settings are the same for all devices, so you just have to do this once.
Then you create a proxy to the remote device. This proxy is a python object that behaves exactly like the device you are connecting to. You can call methods and access properties as if the device was local.

In [ ]:
# Some stuff we will need later on
import time
import matplotlib.pyplot as plt

import Pyro4

camera = Pyro4.Proxy("PYRO:SimulatedCamera@camera_0:8000")

light_source = Pyro4.Proxy("PYRO:SimulatedLightSource@light_0:9000")

## Using devices

### Predefined properties

Each device type (Camera, LightSource, Stage,...) has a set of ***predefined properties*** and methods/functions that are going to be shared between any device of that type.

For example a light source can be turned on and off and can change its power output.

In [ ]:
# Checking the status of a light source
light_source.enabled

Checking the status of the laser returns a very convenient and "pythonic" ```False```. It is a good thing that lasers are off when we connect to them.

Let us turn it on 

In [ ]:
light_source.enable()
light_source.enabled

Let's now change the power of the light source. But first lets turn it off.

In [ ]:
print("Turning off light source")
light_source.disable()
print("Light source power is:")
light_source.power

Light source powers are expressed relative to their nominal power. as values between 0 and 1.

In [ ]:
print("Setting the power at 75 %")
light_source.power = 0.75
print(f"Current power is: {light_source.power * 100} %")
print("Oops! We have to turn on the light...")
light_source.enable()
print(f"Current power is: {light_source.power * 100} %")
time.sleep(1)
light_source.disable()
print("Shutting down...")
light_source.shutdown()
print(f"Lights on? {light_source.get_is_enabled()}")

Cameras have a different set of properties, like **exposure time** or the ROI.

In [ ]:
camera.enable()
print(f"exposure time: {camera.get_exposure_time()} seconds")
camera.set_exposure_time(.2)
print(f"exposure time: {camera.get_exposure_time()} seconds")

print(f"ROI: {camera.get_roi()}")
camera.set_roi((128, 128, 256, 256))
print(f"ROI: {camera.get_roi()}")
camera.set_setting("display image number", False)

Now we can get an image from the camera

In [ ]:
camera.enable()

image = camera.grab_next_data()

plt.imshow(image[0], cmap="gray")

### Device specific properties

Devices from different manufacturers may have settings that are unique to them. This is more common with complex devices such as cameras.
Our simulated camera has some settings that are unique to a simulated camera, such as what type of image has to be produced.

You may explore all the settings of a camera using the ```describe_settings``` function. 
The output of this function is only useful for computers, so we can prettify it a bit

In [ ]:
for setting in camera.describe_settings():
    print(f"name: {setting[0]}")
    print(f"type: {setting[1]['type']}")
    print(f"values: {setting[1]['values']}")
    print(f"readonly: {setting[1]['readonly']}")
    print(f"cached: {setting[1]['cached']}")
    print()
    

You may access the device specific settings through their name

In [ ]:
print(f"image pattern: {camera.get_setting('image pattern')}")
camera.set_setting('image pattern', 1)
print(f"image pattern: {camera.get_setting('image pattern')}")

In [ ]:
type(camera.get_setting("image pattern"))

Once we are done, we clean after ourselves.

In [ ]:
light_source.shutdown()
camera.shutdown()